Before running the code, you might need to uncomment the next cell in order to import Russian-language NLP-library Razdel (https://github.com/natasha)

In [2]:
#!pip install razdel

In [3]:
import nltk
from nltk import word_tokenize
from sacremoses import MosesTokenizer
import spacy
from razdel import tokenize
from nltk.corpus import stopwords
import stopwordsiso
from stopwordsiso import stopwords


Below we will try several tokenizers on a sample Russian text

In [4]:
test_text = """
Илья Сегалович родился в семье советского геофизика В.И.Сегаловича. В физико-математической школе 
познакомился с будущим сооснователем «Яндекса» Аркадием Воложем. В 1981 г. поступил в Московский 
геологоразведочный институт им.Орджоникидзе, где когда-то обучалось 3 тыс. студентов.
"""

In [5]:
tokenized = {}

In [6]:
#NLTK tokenizer
nltk_tokenized = nltk.word_tokenize(test_text, language='russian')

tokenized['NLTK'] = nltk_tokenized

In [7]:
#Moses tokenizer
mt = MosesTokenizer(lang='ru')
moses_tokenized = mt.tokenize(test_text)
tokenized['Moses'] = moses_tokenized

In [8]:
# spacy tokenizer
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/15.3 MB 3.4 MB/s eta 0:00:05
     ----- ---------------------------------- 2.1/15.3 MB 5.9 MB/s eta 0:00:03
     ---------- ----------------------------- 3.9/15.3 MB 6.9 MB/s eta 0:00:02
     --------------- ------------------------ 6.0/15.3 MB 7.7 MB/s eta 0:00:02
     ---------------------- ----------------- 8.7/15.3 MB 8.8 MB/s eta 0:00:01
     ----------------------------- ---------- 11.3/15.3 MB 9.4 MB/s eta 0:00:01
     ------------------------------------ --- 13.9/15.3 MB 9.8 MB/s eta 0:00:01
     ---------------------------------------- 15.3/15.3 MB 9.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [9]:
nlp = spacy.load('ru_core_news_sm')
doc = nlp(test_text)
spacy_tokenized = [token.text for token in doc]
tokenized['Spacy'] =spacy_tokenized

In [10]:
# razdel tokenizer

tokens = list(tokenize(test_text))
razdel_tokenized = [_.text for _ in tokens]
tokenized['Razdel'] = razdel_tokenized

In [11]:
#comparison
for k, v in tokenized.items():
    print(f'Tokenizer {k}: {v}')

Tokenizer NLTK: ['Илья', 'Сегалович', 'родился', 'в', 'семье', 'советского', 'геофизика', 'В.И.Сегаловича', '.', 'В', 'физико-математической', 'школе', 'познакомился', 'с', 'будущим', 'сооснователем', '«', 'Яндекса', '»', 'Аркадием', 'Воложем', '.', 'В', '1981', 'г.', 'поступил', 'в', 'Московский', 'геологоразведочный', 'институт', 'им.Орджоникидзе', ',', 'где', 'когда-то', 'обучалось', '3', 'тыс.', 'студентов', '.']
Tokenizer Moses: ['Илья', 'Сегалович', 'родился', 'в', 'семье', 'советского', 'геофизика', 'В.И.Сегаловича.', 'В', 'физико-математической', 'школе', 'познакомился', 'с', 'будущим', 'сооснователем', '«', 'Яндекса', '»', 'Аркадием', 'Воложем', '.', 'В', '1981', 'г.', 'поступил', 'в', 'Московский', 'геологоразведочный', 'институт', 'им.Орджоникидзе', ',', 'где', 'когда-то', 'обучалось', '3', 'тыс.', 'студентов', '.']
Tokenizer Spacy: ['\n', 'Илья', 'Сегалович', 'родился', 'в', 'семье', 'советского', 'геофизика', 'В.И.Сегаловича', '.', 'В', 'физико', '-', 'математической', 'шк

We do not see too large difference between the results. All tokenizers handle well "«»" qutation marks, separating them from the word. Nevertheless, only Razdel separates period in the abbreviations, thus allowing to separately consider last names and abbreviated terms (in many cases they will coincide with the stemmed form of the word). We will use it for our preprocessing function.

In [14]:
# in addition to the tokenizer, we will use a rule-based stemmer adapted for the Russian language (https://www.nltk.org/_modules/nltk/stem/snowball.html),
# Russian stopword list from NLTK and a list of punctuation marks from string, adding quotation marks, 
# bringing all the tokens to lower case

from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords

def preprocess(text, remove_stop=True) -> list:
    tokens = [_.text for _ in list(tokenize(text))]
    
    preprocessed =[]

    stemmer = SnowballStemmer("russian") 
    stop = stopwords.words("russian")
    punct = string.punctuation + "«»"
    
    for t in tokens:
        if t in punct:
            continue
        if remove_stop and t.lower() in stop:
            continue
        preprocessed.append(stemmer.stem(t))

    return preprocessed 

In [15]:
#test on doc id ed4af92b-0039-453e-8f25-f359225da8e0

text = 'Сергей Аксёнов сообщил о регистрации в Крыму 247 новых случаев COVID-19 Глава Республики Крым Сергей Аксёнов сообщил на своей официальной странице в социальной сети «ВКонтакте» о регистрации 247 новых случаев коронавирусной инфекции в Крыму.'

preprocess(text)

['серг',
 'аксен',
 'сообщ',
 'регистрац',
 'крым',
 '247',
 'нов',
 'случа',
 'COVID-19',
 'глав',
 'республик',
 'крым',
 'серг',
 'аксен',
 'сообщ',
 'сво',
 'официальн',
 'страниц',
 'социальн',
 'сет',
 'вконтакт',
 'регистрац',
 '247',
 'нов',
 'случа',
 'коронавирусн',
 'инфекц',
 'крым']

This preprocessing was used for implementations of TF-IDF and BM25 and appeared appropriate. Nevertheless, the first tests of query expansion with relevance models revealed that among frequent tokens appended to queries to expand them appeared terms like "—", "–" (punctuation marks that turned out to be common in Russian texts, but not appearing in our punctuation list), "котор" and "эт" that correspond to stemmed versions of Russian words meaning "which" and "this". Such a query expansion lead to performance somewhat worse than on original queries. It showed that a modification of preprocessing function is required.

In [16]:
# let's inspect NLTK Russian stopword list
#from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Liza_N\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stopwords_nltk = stopwords.words("russian")
print(stopwords_nltk)
print(len(stopwords_nltk))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [20]:
# an alternative stopword list for Russian language was found at https://github.com/stopwords-iso/stopwords-iso/blob/master/README.md
stopwords_ru = stopwordsiso.stopwords("ru")
print(stopwords_ru)
print(len(stopwords_ru))

{'тоже', 'ночь', 'меньше', 'тринадцатый', 'всю', 'жизнь', 'опять', 'книга', 'нх', 'некоторый', 'наш', 'пойти', 'восемнадцать', 'спасибо', 'этой', 'собой', 'назад', 'ту', 'зачем', 'твой', 'за', 'любить', 'каждые', 'из', 'вами', 'низко', 'мог', 'вечер', 'десять', 'никуда', 'очень', 'давно', 'женщина', 'русский', 'ждать', 'ли', 'вода', 'сих', 'зато', 'эту', 'имеет', 'такие', 'страна', 'вдали', 'голос', 'деньги', 'а', 'были', 'друг', 'может', 'о', 'белый', 'важная', 'позже', 'год', 'еще', 'вон', 'большой', 'наше', 'они', 'сидеть', 'пол', 'часть', 'без', 'делать', 'во', 'м', 'девять', 'меля', 'две', 'комната', 'этими', 'меня', 'ним', 'тому', 'взгляд', 'отец', 'место', 'надо', 'перед', 'действительно', 'теми', 'так', 'вверх', 'решить', 'ребенок', 'говорить', 'сначала', 'слово', 'тебе', 'восемнадцатый', 'просто', 'нет', 'наконец', 'его', 'вернуться', 'от', 'сеаой', 'пока', 'раз', 'себе', 'однако', 'суть', 'шесть', 'около', 'свой', 'этого', 'уже', 'хороший', 'будь', 'семнадцатый', 'самими', 'б

The alternative stopword list is considerably longer and more comprehensive than the NLTK one. We will upgrade our preprocessing function by replacing the stopword list and including discovered punctuation into the punctuation mark list. 

In [21]:
from razdel import tokenize
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords

def preprocess(text, remove_stop: bool=True) -> list:
    """
    :text: str, text of the corresponding document
    :param remove_stop: bool indicating if stopwords should be removed (default True)

    :return: list(str) of tokens, stemmed, with removed punctuation
    """


    tokens = [_.text for _ in list(tokenize(text))]

    preprocessed =[]

    stemmer = SnowballStemmer("russian")
    stopwords_ru = stopwords("ru")
    punct = string.punctuation + "«»" + "—" + '–'

    for t in tokens:
        if t in punct:
            continue
        if remove_stop and t.lower() in stopwords_ru:
            continue
        preprocessed.append(stemmer.stem(t))

    return preprocessed 

All experiments in CLIR word-overlap based methods were rerun with the updated preprocessing function. For TF-IDF and BM25 there was insignificant increase in performance metrics. At the same time, the quality of expanded queries improved, leading to relevance model based techniques outperform the original BM25 implementation. 